In [1]:
import wrangle
import pandas as pd


In [2]:
df = pd.read_csv("zillow.csv")

/usr/local/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (37,38,63,67,68) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def drop_50_pct_null(df):
    '''This function takes in the zillow df
    removes all columns and rows with 50% nulls or more
    returns df'''
    # Drop columns with 50% or more missing values
    df = df.dropna(axis = 1, thresh = 0.5 * len(df.index))
    # went from 67 columns down to 33
    # drop rows with 50% or more missing vlaues
    df.dropna(axis = 0, thresh = 0.5 * len(df.columns))
        # ended up not dropping any rows 
            # will remain in the function in case anything were to change later on
    return df

def clean_zillow(df):
    '''This function takes in the df
    applies all the cleaning funcitons previously created
    drops columns
    renames columns'''
    # assuming null value for pool means no pool
    df.poolcnt.fillna(0, inplace = True)
    # assuming null calie for fireplace means no null
    df.fireplacecnt.fillna(0, inplace = True)
    # drop features/rows with more than 50% null values
    df = drop_50_pct_null(df)
    # create dummy variables and add them to the df
    dummy_df =  pd.get_dummies(df['fips'])
    dummy_df.columns = ['in_los_angeles', 'in_orange_county', 'in_ventura']
    df = pd.concat([df, dummy_df], axis=1)
    df['fips'] = df.fips.replace('6,037.00', 6037)
    df['fips'] = df.fips.replace('6,059.00', 6059)
    df['fips'] = df.fips.replace('6,111.00', 6111)
    #create new feature house_age
    today = pd.to_datetime('today')
    df['house_age'] = today.year - df['yearbuilt']
    #create new feature tax_rate which is the monthyl taxes
    df['tax_rate'] = df.taxvaluedollarcnt / df.taxamount
    # create new feature for log_error_levels
    df['level_of_log_error'] = pd.qcut(df.logerror, q=5, labels=['L1', 'L2', 'L3', 'L4', 'L5'])
    # create new feature acres
    df['acres'] = df.lotsizesquarefeet/43560
    #drop features
    df = df.drop(['propertycountylandusecode', 'propertyzoningdesc', 
                 'heatingorsystemdesc', 'transactiondate',
                  'finishedsquarefeet12', 'id', 'censustractandblock',
                 'rawcensustractandblock', 'calculatedbathnbr', 
                 'assessmentyear'], axis=1)
    #rename features
    df = df.rename(columns={'heatingorsystemtypeid':'has_heating_system', 
                           'bathroomcnt':'bathrooms', 'bedroomcnt':'bedrooms', 
                           'buildingqualitytypeid':'quality',
                           'calculatedfinishedsquarefeet':'square_feet', 
                           'fullbathcnt':'full_bathrooms',
                           'lotsizesquarefeet':'lot_square_feet', 
                           'propertylandusetypeid':'land_type',
                           'regionidcity':'city', 'regionidcounty':'county',
                           'regionidzip':'zip_code', 'roomcnt':'room_count',
                           'structuretaxvaluedollarcnt':'structure_tax_value',
                           'taxvaluedollarcnt':'tax_value', 
                           'landtaxvaluedollarcnt':'land_tax_value', 
                           'fireplacecnt':'has_fireplace',
                           'poolcnt':'has_pool'})
    # assuming that null means no heating bc it is southern CA
    df.has_heating_system.fillna('13', inplace = True)
    # change has_heating_system to binary
    df['has_heating_system'] = df.has_heating_system.replace([2.0, 7.0, 24.0, 6.0, 20.0, 13.0, 18.0, 1.0, 10.0, 11.0], 1)
    df['has_heating_system'] = df.has_heating_system.replace('13', '0')
    # drop all remaining null values
    # all of these are 1 unit counts
    df.unitcnt.fillna(1, inplace = True)
    df['unitcnt'] = df.unitcnt.replace([2.0, 3.0, 4.0, 6.0], 1)
    # replacing null in quality feature with its median range (6)
    df.quality.fillna(6.0, inplace = True)
    # replacing null in square_feet with its median
    df.lot_square_feet.fillna(7313, inplace = True)
     # replacing null in quality feature with its median
    df.square_feet.fillna(1511, inplace = True)
     # replacing null in quality feature with its median
    df.full_bathrooms.fillna(2, inplace = True)
     # replacing null in quality feature with its median
    df.yearbuilt.fillna(1970, inplace = True)
     # replacing null in quality feature with its median
    df.structure_tax_value.fillna(134871, inplace = True)
     # replacing null in quality feature with its median
    df.house_age.fillna(51, inplace = True)
     # replacing null in quality feature with its median
    df.city.fillna(25218, inplace = True)
     # replacing null in quality feature with its median
    df.zip_code.fillna(96410, inplace = True)
    # change has_fireplace to a binary
    df['has_fireplace'] = df.has_fireplace.replace([2.0, 3.0, 4.0, 5.0], 1)
    df['has_fireplace'] = df.has_fireplace.replace(0.0, 0)
    #fix has_pool to int
    df['has_pool'] = df.has_fireplace.replace(1.0, 1)
    df['has_pool'] = df.has_fireplace.replace(0.0, 0)
    # change unitcnt and has_heating_system to integers istead of objects
    df['has_heating_system'] = (df['has_heating_system'] == True ).astype(int)
    df['unitcnt'] = (df['unitcnt'] == True ).astype(int)
    #drop remaining null values
    df = df.dropna()
    # change la, oc, and vent into int
    df['in_los_angeles'] = (df['in_los_angeles'] == True ).astype(int)
    df['in_orange_county'] = (df['in_orange_county'] == True ).astype(int)
    df['in_ventura'] = (df['in_ventura'] == True ).astype(int)
    # set index as parcelid
    df = df.set_index('parcelid')
    # drop random column that came up
    df = df.drop(['Unnamed: 0'], axis=1)
    return df
df = clean_zillow(df)

In [4]:
df.isnull().sum()

land_type              0
has_heating_system     0
bathrooms              0
bedrooms               0
quality                0
square_feet            0
fips                   0
has_fireplace          0
full_bathrooms         0
latitude               0
longitude              0
lot_square_feet        0
has_pool               0
city                   0
county                 0
zip_code               0
room_count             0
unitcnt                0
yearbuilt              0
structure_tax_value    0
tax_value              0
land_tax_value         0
taxamount              0
logerror               0
propertylandusedesc    0
in_los_angeles         0
in_orange_county       0
in_ventura             0
house_age              0
tax_rate               0
level_of_log_error     0
acres                  0
dtype: int64

In [5]:
df.head()

,land_type,has_heating_system,bathrooms,bedrooms,quality,square_feet,fips,has_fireplace,full_bathrooms,latitude,...,taxamount,logerror,propertylandusedesc,in_los_angeles,in_orange_county,in_ventura,house_age,tax_rate,level_of_log_error,acres
parcelid,,,,,,,,,,,,,,,,,,,,,
10711855,261.0,1,2.0,3.0,8.0,2107.0,6037.0,0.0,2.0,34222559.0,...,7659.36,-0.007357,Single Family Residential,1,0,0,49.0,81.487096,L2,0.210239
10711877,261.0,1,2.0,4.0,8.0,1882.0,6037.0,0.0,2.0,34220261.0,...,8123.91,0.021066,Single Family Residential,1,0,0,49.0,81.241668,L4,0.207415
10711888,261.0,1,2.0,4.0,8.0,1882.0,6037.0,0.0,2.0,34222491.0,...,6673.24,0.077174,Single Family Residential,1,0,0,49.0,81.358231,L5,0.224977
10711910,261.0,1,2.0,3.0,8.0,1477.0,6037.0,0.0,2.0,34221864.0,...,1116.46,-0.041238,Single Family Residential,1,0,0,61.0,69.891443,L1,0.259068
10711923,261.0,1,2.0,4.0,8.0,1918.0,6037.0,0.0,2.0,34220619.0,...,5239.85,-0.009496,Single Family Residential,1,0,0,61.0,79.288338,L2,0.258012
